<a href="https://colab.research.google.com/github/ailuj/BayesianDeepLearning/blob/paper/Paper_all_results_det_regularized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/data_sets"


test_set_40k.csv    train_set_150k.csv	train_set_240k.csv
train_set_100k.csv  train_set_200k.csv	train_set_60k.csv


In [0]:
import pandas as pd
test_40k = pd.read_csv('/content/drive/My Drive/data_sets/test_set_40k.csv')
train_60k = pd.read_csv('/content/drive/My Drive/data_sets/train_set_60k.csv')
train_100k = pd.read_csv('/content/drive/My Drive/data_sets/train_set_100k.csv')
train_150k = pd.read_csv('/content/drive/My Drive/data_sets/train_set_150k.csv')
train_200k = pd.read_csv('/content/drive/My Drive/data_sets/train_set_200k.csv')
train_240k = pd.read_csv('/content/drive/My Drive/data_sets/train_set_240k.csv')

In [0]:
full = train_240k.append(test_40k)

In [0]:
full = full[['int_rate','loan_amnt','purpose', 'profit_rate']]
full_features = full[['int_rate','loan_amnt','purpose']]
full_features = pd.get_dummies(full_features)
full_target = full[['profit_rate']]

In [0]:
def preprocess(x):
  from sklearn import preprocessing
  x = x.dropna()
 
  x_features = x[['int_rate','loan_amnt','purpose']]
  x_features = pd.get_dummies(x_features)
  x_target = x[['profit_rate']]

  scaler_features = preprocessing.MinMaxScaler().fit(full_features)
  x_features = scaler_features.transform(x_features)
  
  scaler_target = preprocessing.MinMaxScaler().fit(full_target)
  x_target = scaler_target.transform(x_target)
  
  return (x_features, x_target)

In [0]:
((test_40k_features,test_40k_target),
 (train_60k_features,train_60k_target),
 (train_100k_features,train_100k_target),
 (train_150k_features,train_150k_target),
 (train_200k_features,train_200k_target),
 (train_240k_features,train_240k_target)) = (preprocess(test_40k), 
                                             preprocess(train_60k), 
                                             preprocess(train_100k), 
                                             preprocess(train_150k), 
                                             preprocess(train_200k), 
                                             preprocess(train_240k)) 

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype u

In [0]:
((test_40k_features.shape,test_40k_target.shape),
 (train_60k_features.shape,train_60k_target.shape),
 (train_100k_features.shape,train_100k_target.shape),
 (train_150k_features.shape,train_150k_target.shape),
 (train_200k_features.shape,train_200k_target.shape),
 (train_240k_features.shape,train_240k_target.shape))

(((40000, 16), (40000, 1)),
 ((65600, 16), (65600, 1)),
 ((105600, 16), (105600, 1)),
 ((155600, 16), (155600, 1)),
 ((205600, 16), (205600, 1)),
 ((245600, 16), (245600, 1)))

# Modeling


In [0]:
from keras import Model as mcModel
from keras import Input as mcInput
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.regularizers import l2

def architecture(model_type, h_cells, n_hidden, input_dim, dropout_prob, reg):
   
  if model_type == 'deterministic_regularized':
    from keras.models import Sequential 
    from keras.layers import Dropout 
    from keras.layers import Dense 
    
    model2 = Sequential()
    model2.add(Dropout(dropout_prob, input_shape=(input_dim,)))
    model2.add(Dense(h_cells, activation='relu', W_regularizer=l2(reg)))
    model2.add(Dropout(dropout_prob))
    model2.add(Dense(h_cells, activation='relu', W_regularizer=l2(reg)))
    model2.add(Dropout(dropout_prob))
    model2.add(Dense(1))
    return model2

def model_runner(X_train, y_train, X_test=test_40k_features, y_test=test_40k_target,
                 dropout_prob=0.20, n_epochs=100, tau=1.0, batch_size=500, 
                 lengthscale=1e-2, n_hidden=[100,100], h_cells=100):
  
  input_dim = X_train.shape[1]
  N = X_train.shape[0]
  reg = lengthscale**2 * (1 - dropout_prob) / (2. * N * tau)

  
  print('Regularized Deterministic NN fit')

  model_det_regularized = architecture(model_type = 'deterministic_regularized', 
                                   h_cells= h_cells, n_hidden=n_hidden, input_dim=input_dim, 
                                  dropout_prob=dropout_prob, reg=reg)
 
  model_det_regularized.compile(optimizer='adam', loss='mse', metrics=['mae'])
  model_det_regularized.fit(X_train, y_train, batch_size=batch_size, nb_epoch=n_epochs, verbose=1)
 
  
  
  return model_det_regularized


def predictor(model_det_regularized, 
              X_test=test_40k_features, y_test = test_40k_target, T = 1000):
  from sklearn.metrics import mean_squared_error
  import numpy as np
  
  
  pred_det_regularized = model_det_regularized.predict(X_test, verbose=1)
  mse_det_regularized = mean_squared_error(pred_det_regularized, y_test)
  print(mse_det_regularized)
        
  
  return pred_det_regularized

In [0]:
model_det_regularized_60k = model_runner(train_60k_features, train_60k_target)
pred_det_regularized_60k = predictor( model_det_regularized_60k)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`


Regularized Deterministic NN fit


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
65600/65600 [==============================] - 1s 19us/step - loss: 0.0674 - mean_absolute_error: 0.2049
Epoch 2/100
65600/65600 [==============================] - 1s 11us/step - loss: 0.0513 - mean_absolute_error: 0.1790
Epoch 3/100
65600/65600 [==============================] - 1s 11us/step - loss: 0.0503 - mean_absolute_error: 0.1770
Epoch 4/100
65600/65600 [==============================] - 1s 11us/step - loss: 0.0498 - mean_absolute_error: 0.1763
Epoch 5/100
65600/65600 [==============================] - 1s 11us/step - loss: 0.0494 - mean_absolute_error: 0.1751
Epoch 6/100
65600/65600 [==============================] - 1s 11us/step - loss: 0.0493 - mean_absolute_error: 0.1750
Epoch 7/100
65600/65600 [==============================] - 1s 11us/step - loss: 0.0491 - mean_absolute_error: 0.1747
Epoch 8/100
65600/65600 [==============================] - 1s 11us/step - loss: 0.0489 - mean_absolute_error: 0.1743
Epoch 9/100
65600/65600 [==============================] - 1s 11

In [0]:


import pickle
outfile = open('pred_det_regularized_60k.pickle','wb')
pickle.dump(pred_det_regularized_60k,outfile)
outfile.close()



from google.colab import files

files.download('pred_det_regularized_60k.pickle')


In [0]:
model_det_regularized_100k = model_runner(train_100k_features, train_100k_target)
pred_det_regularized_100k = predictor(model_det_regularized_100k)

Regularized Deterministic NN fit


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
105600/105600 [==============================] - 2s 16us/step - loss: 0.0621 - mean_absolute_error: 0.1962
Epoch 2/100
105600/105600 [==============================] - 1s 11us/step - loss: 0.0504 - mean_absolute_error: 0.1770
Epoch 3/100
105600/105600 [==============================] - 1s 11us/step - loss: 0.0496 - mean_absolute_error: 0.1753
Epoch 4/100
105600/105600 [==============================] - 1s 11us/step - loss: 0.0490 - mean_absolute_error: 0.1743
Epoch 5/100
105600/105600 [==============================] - 1s 11us/step - loss: 0.0488 - mean_absolute_error: 0.1741
Epoch 6/100
105600/105600 [==============================] - 1s 11us/step - loss: 0.0488 - mean_absolute_error: 0.1740
Epoch 7/100
105600/105600 [==============================] - 1s 11us/step - loss: 0.0486 - mean_absolute_error: 0.1735
Epoch 8/100
105600/105600 [==============================] - 1s 11us/step - loss: 0.0484 - mean_absolute_error: 0.1731
Epoch 9/100
105600/105600 [=====================

In [0]:


import pickle
outfile = open('pred_det_regularized_100k.pickle','wb')
pickle.dump(pred_det_regularized_100k,outfile)
outfile.close()


from google.colab import files

files.download('pred_det_regularized_100k.pickle')


In [0]:
model_det_regularized_150k = model_runner(train_150k_features, train_150k_target)
pred_det_regularized_150k = predictor(model_det_regularized_150k)

Regularized Deterministic NN fit


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
155600/155600 [==============================] - 2s 15us/step - loss: 0.0584 - mean_absolute_error: 0.1904
Epoch 2/100
155600/155600 [==============================] - 2s 11us/step - loss: 0.0498 - mean_absolute_error: 0.1764
Epoch 3/100
155600/155600 [==============================] - 2s 11us/step - loss: 0.0493 - mean_absolute_error: 0.1754
Epoch 4/100
155600/155600 [==============================] - 2s 11us/step - loss: 0.0489 - mean_absolute_error: 0.1745
Epoch 5/100
155600/155600 [==============================] - 2s 11us/step - loss: 0.0487 - mean_absolute_error: 0.1741
Epoch 6/100
155600/155600 [==============================] - 2s 11us/step - loss: 0.0485 - mean_absolute_error: 0.1736
Epoch 7/100
155600/155600 [==============================] - 2s 11us/step - loss: 0.0484 - mean_absolute_error: 0.1736
Epoch 8/100
155600/155600 [==============================] - 2s 11us/step - loss: 0.0483 - mean_absolute_error: 0.1733
Epoch 9/100
155600/155600 [=====================

In [0]:


import pickle
outfile = open('pred_det_regularized_150k.pickle','wb')
pickle.dump(pred_det_regularized_150k,outfile)
outfile.close()

from google.colab import files

files.download('pred_det_regularized_150k.pickle')


In [0]:
model_det_regularized_200k= model_runner(train_200k_features, train_200k_target)
pred_det_regularized_200k= predictor(model_det_regularized_200k)

Regularized Deterministic NN fit


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
205600/205600 [==============================] - 3s 14us/step - loss: 0.0557 - mean_absolute_error: 0.1861
Epoch 2/100
205600/205600 [==============================] - 2s 11us/step - loss: 0.0493 - mean_absolute_error: 0.1752
Epoch 3/100
205600/205600 [==============================] - 2s 11us/step - loss: 0.0488 - mean_absolute_error: 0.1741
Epoch 4/100
205600/205600 [==============================] - 2s 11us/step - loss: 0.0485 - mean_absolute_error: 0.1736
Epoch 5/100
205600/205600 [==============================] - 2s 11us/step - loss: 0.0484 - mean_absolute_error: 0.1735
Epoch 6/100
205600/205600 [==============================] - 2s 11us/step - loss: 0.0482 - mean_absolute_error: 0.1730
Epoch 7/100
205600/205600 [==============================] - 2s 11us/step - loss: 0.0481 - mean_absolute_error: 0.1729
Epoch 8/100
205600/205600 [==============================] - 2s 11us/step - loss: 0.0480 - mean_absolute_error: 0.1729
Epoch 9/100
205600/205600 [=====================

In [0]:

import pickle
outfile = open('pred_det_regularized_200k.pickle','wb')
pickle.dump(pred_det_regularized_200k,outfile)
outfile.close()


from google.colab import files

files.download('pred_det_regularized_200k.pickle')


In [0]:
model_det_regularized_240k = model_runner(train_240k_features, train_240k_target)
pred_det_regularized_240k = predictor(model_det_regularized_240k)

Regularized Deterministic NN fit


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, activation="relu", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/100
245600/245600 [==============================] - 4s 15us/step - loss: 0.0545 - mean_absolute_error: 0.1837
Epoch 2/100
245600/245600 [==============================] - 3s 11us/step - loss: 0.0491 - mean_absolute_error: 0.1744
Epoch 3/100
245600/245600 [==============================] - 3s 11us/step - loss: 0.0486 - mean_absolute_error: 0.1736
Epoch 4/100
245600/245600 [==============================] - 3s 11us/step - loss: 0.0483 - mean_absolute_error: 0.1731
Epoch 5/100
245600/245600 [==============================] - 3s 11us/step - loss: 0.0481 - mean_absolute_error: 0.1726
Epoch 6/100
245600/245600 [==============================] - 3s 11us/step - loss: 0.0480 - mean_absolute_error: 0.1727
Epoch 7/100
245600/245600 [==============================] - 3s 11us/step - loss: 0.0478 - mean_absolute_error: 0.1723
Epoch 8/100
245600/245600 [==============================] - 3s 11us/step - loss: 0.0478 - mean_absolute_error: 0.1723
Epoch 9/100
245600/245600 [=====================

In [0]:


import pickle
outfile = open('pred_det_regularized_240k.pickle','wb')
pickle.dump(pred_det_regularized_240k,outfile)
outfile.close()



from google.colab import files

files.download('pred_det_regularized_240k.pickle')
